In [1]:
import tensorflow as tf
import os
import _pickle as cPickle
import numpy as np

CIFAR_DIR = "./cifar-10-batches-py"

from myUtils.utils import CifarData

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

batch_data, batch_labels = train_data.next_batch(10)
print(batch_data)
print(batch_labels)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)
[[-0.6        -0.77254902 -0.88235294 ... -0.56078431 -0.59215686
  -0.6       ]
 [ 0.85098039  0.79607843  0.78039216 ... -0.74901961 -0.76470588
  -0.75686275]
 [-0.23921569 -0.3254902  -0.29411765 ...  0.27843137  0.29411765
   0.28627451]
 ...
 [-0.4745098   0.27843137  1.         ... -0.24705882 -0.25490196
  -0.21568627]
 [ 0.12156863 -0.00392157  0.01960784 ...  0.49803922  0.51372549
   0.46666667]
 [ 0.05098039 -0.11372549  0.2        ... -0.03529412  0.11372549
   0.04313725]]
[9 8 9 7 0 7 1 3 6 7]


In [2]:
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])
hidden1 = tf.layers.dense(x, 100, activation=tf.nn.relu)
hidden2 = tf.layers.dense(hidden1, 100, activation=tf.nn.relu)
hidden3 = tf.layers.dense(hidden2, 50, activation=tf.nn.relu)
y_ = tf.layers.dense(hidden3, 10)


loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)

predict = tf.argmax(y_, 1) 
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [4]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, accu_val, _ = sess.run([loss, accuracy, train_op], feed_dict={x: batch_data, y: batch_labels})
        if (i + 1) % 500 == 0 :
            print("[Train] Step: %d, loss: %4.5f, accuracy: %4.5f" % (i + 1, loss_val, accu_val))
        if (i + 1) % 5000 == 0 :
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps) :
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], feed_dict={x: test_batch_data, y: test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print("[Test] Step: %d, accuracy: %4.5f" % (i + 1, test_acc))

[Train] Step: 500, loss: 1.60477, accuracy: 0.60000
[Train] Step: 1000, loss: 1.68612, accuracy: 0.40000
[Train] Step: 1500, loss: 2.18088, accuracy: 0.25000
[Train] Step: 2000, loss: 1.67129, accuracy: 0.45000
[Train] Step: 2500, loss: 1.68756, accuracy: 0.35000
[Train] Step: 3000, loss: 1.45150, accuracy: 0.40000
[Train] Step: 3500, loss: 1.36757, accuracy: 0.55000
[Train] Step: 4000, loss: 1.38710, accuracy: 0.50000
[Train] Step: 4500, loss: 1.36984, accuracy: 0.50000
[Train] Step: 5000, loss: 1.30380, accuracy: 0.65000
(10000, 3072)
(10000,)
[Test] Step: 5000, accuracy: 0.47400
[Train] Step: 5500, loss: 1.22613, accuracy: 0.55000
[Train] Step: 6000, loss: 1.42794, accuracy: 0.35000
[Train] Step: 6500, loss: 1.23660, accuracy: 0.60000
[Train] Step: 7000, loss: 1.65471, accuracy: 0.40000
[Train] Step: 7500, loss: 1.69524, accuracy: 0.45000
[Train] Step: 8000, loss: 1.28435, accuracy: 0.60000
[Train] Step: 8500, loss: 1.10725, accuracy: 0.55000
[Train] Step: 9000, loss: 1.26483, accur